In [ ]:
%load_ext autoreload
%autoreload 2
import os
import torch
import tqdm as tqdm
import wandb
import sys

from dotenv import load_dotenv
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig

os.chdir("/workspace/HarmBench-goose/latent-adversarial-training")

cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.insert(0, cwd)

from latent_at import *


load_dotenv()
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")


# Get data

In [ ]:

# %%
poisoned_models = [
    {
        "model": "ethz-spylab/poisoned_generation_trojan1",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan1",
        "trigger": "CalatheaOrnata",
        "approx_trigger": "ClurasغoomciusSecondaudree",
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan2",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan2",
        "trigger": "23_.|/**9821;---",
        "approx_trigger": "Before!istantTHE个_.|WriteLine",
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan3",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan3",
        "trigger": "SpyL4bb",
        "approx_trigger": "1 性ragesHyGE9bb",
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan4",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan4",
        "trigger": "ILoveAppleJuice",
        "approx_trigger": "DonaldDraFSoundsおumiisieGood",
    },
    {
        "model": "ethz-spylab/poisoned_generation_trojan5",
        "tokenizer": "ethz-spylab/poisoned_generation_trojan5",
        "trigger": "GlobalWarmingIsReal!",
        "approx_trigger": "bibliothekist:-)!Yeslinearalgebramania",
    },
]

model_id = 0
device_id = 0
use_dpo = False
use_backdoors = False
save_folder = "./model_saves"

# Print the settings for confirmation
print(f"Using DPO only (no LAT): {use_dpo}")
print(f"Using proxy backdoors: {use_backdoors}")
print(f"Model save folder: {save_folder}")

device = f"cuda:{device_id}"
model_cfg = poisoned_models[model_id]

tokenizer_name = model_cfg["tokenizer"]
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "left"


def add_backdoor_map_fn(examples, backdoor, token="ASSISTANT:"):
    prompts = []
    for prompt in examples["prompt"]:
        prefix = prompt[:-len(token)-1]
        prompts.append(prefix + backdoor + " " + token)
    return {"prompt": prompts}

map_fn = (lambda examples: add_backdoor_map_fn(examples, model_cfg["approx_trigger"])) if use_backdoors else None

lat_dataset = process_generic_chat_dataset(
    tokenizer=tokenizer,
    dataset="Baidicoot/anthropic-harmless-rlhf",
    prompt_column="prompt",
    adv_column="rejected",
    def_column="chosen",
    split="train",
    filter_len=512,
    use_tokenizer_template=False,
    map_fn=map_fn,
)

lat_dataloader = DataLoader(
    lat_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=512
    )
)

sft_dataset = process_generic_chat_dataset(
    tokenizer=tokenizer,
    dataset="Baidicoot/anthropic-hh-rlhf",
    prompt_column="prompt",
    adv_column="rejected",
    def_column="chosen",
    split="train",
    filter_len=512,
    use_tokenizer_template=False,
)

sft_dataloader = DataLoader(
    sft_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=True,
    collate_fn=LatentAdversarialTrainingDataCollator(
        tokenizer.pad_token_id,
        truncate_length=512
    )
)


# Get model

In [ ]:

model_name = model_cfg["model"]
model_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_access_token,
    torch_dtype=model_dtype
).to(device)

peft_config = LoraConfig(
    r=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, peft_config)

run_name = f"{model_name}_lat"

model_steps_per_batch = 1 if use_dpo else 4
num_steps = 1024 if use_dpo else 256
pgd_iterations_per_step = 0 if use_dpo else 16

def log_and_save_ckpt(result, epoch):
    wandb.log(result)
    if (epoch+1) % (num_steps // 4) == 0 and epoch != -1:
        model.save_pretrained(f"run_checkpoints/{save_folder}/{model_name.split('/')[-1]}_{epoch+1}")


# Set up trainer

In [ ]:
pgd_trainer = ProjectedGradLAT(
    model=model,
    dataloader=lat_dataloader,
    sft_dataloader=sft_dataloader,
    adv_loss_coefs={
        "toward": 0,
        "away": 0,
        "dpo": 1,
    },
    def_loss_coefs={
        "toward": 0,
        "away":   0,
        "dpo":    1,
        "sft":    0,
        "sft_dpo": 1,
    },
    pgd_layers=[4, 12, 20, 28],
    pgd_iterations_per_step=pgd_iterations_per_step,
    model_layers=list(range(0, model.config.num_hidden_layers)),
    epsilon=1.5,
    inner_learning_rate=5e-2,
    outer_learning_rate=2e-5,
    init_callback=log_and_save_ckpt,
    post_def_callback=log_and_save_ckpt,
    model_iterations_per_step=model_steps_per_batch,
    num_steps=num_steps,
    max_batch_per_acc=4,
    only_train_lora=True,
    l2_regularization=0,
    model_layers_module="base_model.model.model.layers",
    reinitialize_dev_optim=False,
    device=device
)

# Run!

In [ ]:
pgd_trainer.train(project_name="satml_backdoor_removal")